In [481]:
import os
import json
import math

In [482]:
inPath = "./train"
testPath = "./test"
fileExtension = ".txt"
outPathText = "./Processed_text"
outPathJson = "./Output"
word_count = {}
unigram_count = {}
bigram_count = {}
trigram_count = {}
lambdas = []
minLambda = []
lambdas.append([0.4,0.3,0.3])
lambdas.append([0.3,0.4,0.3])
lambdas.append([0.1,0.7,0.2])
lambdas.append([0.5,0.4,0.3])
lambdas.append([0.1,0.4,0.5])
lambdas.append([0.7,0.2,0.1])
lambdas.append([0.8,0.1,0.1])
lambdas.append([0.1,0.8,0.1])
lambdas.append([0.1,0.2,0.8])



In [483]:
def initialize_dicts():
    word_count = {}
    unigram_count = {}
    bigram_count = {}
    trigram_count = {}

In [484]:
def createUnigrams(wordList,outDict):
    for i in range(0, len(wordList)):
        unigram = wordList[i]
        if unigram in outDict:
            outDict[unigram] = outDict[unigram] + 1
        else:
            outDict[unigram] = 1

In [485]:
def createBigrams(wordList,outDict):
    for i in range(0, len(wordList) - 1, 2):
        bigram = wordList[i] + " " + wordList[i + 1]
        if bigram in outDict:
            outDict[bigram] = outDict[bigram] + 1
        else:
            outDict[bigram] = 1

In [486]:
def createTrigrams(wordList,outDict):
    for i in range(0, len(wordList) - 2, 3):
        trigram = wordList[i] + " " + wordList[i + 1] + " " + wordList[i+2]
        if trigram in outDict:
            outDict[trigram] = outDict[trigram] + 1
        else:
            outDict[trigram] = 1

In [487]:
def writeToTxt(text, path, fileName, mode):
    fout = open(path + "/" + fileName, mode)
    print("Writing to file  " + fileName + " at " + path)
    fout.write(text)
    fout.close()

In [488]:
def get_word_count(wordList,outDict):
    for word in wordList:
        if word in outDict:
            outDict[word] = outDict[word] + 1
        else:
            outDict[word] = 1

In [489]:
def get_word_list(path, fileName):
    fp = open(path + "/" + fileName, "r+")
    print("Getting word count...." + fileName)
    text = fp.read()
    text = text.replace("\n\n", " </s> <s> ")
    text = text.replace("."," . ")
    text = text.replace(","," , ")
    text = text.replace("?"," ? ")
    text = text.strip().strip()
    text = " ".join(text.split()).lower()
    wordList = text.split()
    return wordList

In [498]:
def process_file_train(wordList):
    processed_text = ""
    for word in wordList:
        if word_count[word] <= 3:
            processed_text = processed_text + "UNK "
        else:
            processed_text = processed_text + word + " "
    writeToTxt(processed_text, outPathText, file, "w+")
    return processed_text

In [500]:
def process_file_test(wordList):
    processed_text = ""
    for word in wordList:
        if word not in unigram_count:
            processed_text = processed_text + "UNK "
        elif unigram_count[word] <= 3:
            processed_text = processed_text + "UNK "
        else:
            processed_text = processed_text + word + " "
#     writeToTxt(processed_text,outPathText,"heldOut_"+ file, "w+")
    return processed_text

In [492]:
def generate_ngram_json2txt():
    for file in os.listdir(outPathJson):
        if file.endswith("json"):
            with open("./Output/" + file) as json_f:
                print("Reading file... " + file)
                data = json.load(json_f)
                nGrams = ""
                for k, v in sorted(data.items(), key=lambda item: item[1], reverse=True):
                    nGrams = nGrams + k + " : " + str(v) + "\n"
                writeToTxt(nGrams, outPathJson, file.replace("_c","C").replace("json","txt"), "w+")


In [493]:
def generate_ngram_txt(data,name):
    nGrams = ""
    for k, v in sorted(data.items(), key=lambda item: item[1], reverse=True):
        nGrams = nGrams + k + " : " + str(v) + "\n"
    writeToTxt(nGrams, outPathJson,name + ".txt" , "w+")

In [494]:
def calculate_perplexity(test_text,N,lambdas):
    l1 = lambdas[0]
    l2 = lambdas[1]
    l3 = lambdas[2]
    test_text = test_text.strip()
    wordList = test_text.split(" ")
    wordList = filter(lambda x: len(x) != 0, wordList) 
    totalWords = 0
    log_trigram_prob = 0
    for i in range(0,len(wordList)-3):
        trigram_prob = 0
        bigram_prob = 0
        unigram_prob = 0
        n0 = wordList[i]
        n1 = wordList[i+1]
        n2 = wordList[i+2]
#         if n2 not in unigram_count:
#             print(wordList[i-1] + "|"+n0 +"|"+ n1 +"|"+n2+"|" + wordList[i+3] +"|" + wordList[i+4] + " " + wordList[i+5])
#             return -1;
        unigram_prob = float(unigram_count[n2])/N
        if n1 + " " + n2 in bigram_count:
            bigram_prob = float(bigram_count[n1+ " " + n2]) / unigram_count[n1]
        else :
            bigram_prob = 0
        if n0 + " " + n1 + " " + n2 in trigram_count and n0 + " " + n1 in bigram_count:
            trigram_prob = float(trigram_count[n0 + " " + n1 + " " + n2]) / bigram_count[n0 + " " + n1]
        else:
            trigram_prob = 0
#         print("|"+n0 +"|"+ n1 +"|"+n2+"|"+ str(l1*unigram_prob) + " "+ str(l2*bigram_prob) + " " + str(l3*trigram_prob))
        log_trigram_prob += math.log(l1*trigram_prob + l2*bigram_prob + l3*unigram_prob,2)
        totalWords += 1
    log_perplexity = -1 * log_trigram_prob / totalWords
#     print("log_trigram_prob " + str(log_trigram_prob) + " " + str(totalWords))
    perplexity = math.pow(2,log_perplexity)
    print("lambda: " + str(lambdas[0]) + " " + str(lambdas[1]) + " " + str(lambdas[2]) + " " + "Perplexity :" + str(perplexity))
    return perplexity

In [495]:
# def main():
#     i = 0
def grid_search():
    heldOut = 1
    fileList = os.listdir(inPath)
    for heldOut in range(9,len(fileList)-1):
        initialize_dicts()
        heldOutFileList = fileList[heldOut:heldOut+2]
        for file in fileList:
            if file.endswith(fileExtension) and file not in heldOutFileList:
                wordList = get_word_list(file)
                get_word_count(wordList,word_count)
                processed_text = process_file_train(wordList)
                wordList_UNK = processed_text.split()
                createUnigrams(wordList_UNK,unigram_count)
                createBigrams(wordList_UNK,bigram_count)
                createTrigrams(wordList_UNK,trigram_count)

        print("========================")
        print("Testing on held out data .......")
        print("========================")
        test_text = ""
        for f in heldOutFileList:
            print(f)
            if f.endswith(fileExtension):
                wordList = get_word_list(f)
                test_text = test_text + process_file_test(wordList)
        minPerplexity = 0
        for l in lambdas:
            pp = calculate_perplexity(test_text,sum(unigram_count.values()),l)
            if pp < minPerplexity:
                minPerplexity = pp
                minLambda = l
        return l
            

In [464]:
maxLam = grid_search()

Getting word count....9.txt
Getting word count....8.txt
Getting word count....5.txt
Getting word count....4.txt
Getting word count....6.txt
Getting word count....7.txt
Getting word count....3.txt
Getting word count....2.txt
Testing on held out data .......
0.txt
Getting word count....0.txt
1.txt
Getting word count....1.txt
lambda: 0.4 0.3 0.3 Perplexity :164.885799709
lambda: 0.3 0.4 0.3 Perplexity :161.912968675
lambda: 0.1 0.7 0.2 Perplexity :171.350439678
lambda: 0.5 0.4 0.3 Perplexity :137.521071543
lambda: 0.1 0.4 0.5 Perplexity :176.646946205
lambda: 0.7 0.2 0.1 Perplexity :203.917143415
lambda: 0.8 0.1 0.1 Perplexity :235.985725038
lambda: 0.1 0.8 0.1 Perplexity :182.277314697
lambda: 0.1 0.2 0.8 Perplexity :187.304344044


In [496]:
def trainLM():
    fileList = os.listdir(inPath)
    initialize_dicts()
    for file in fileList:
        if file.endswith(fileExtension):
            wordList = get_word_list(inPath,file)
            get_word_count(wordList,word_count)
            processed_text = process_file_train(wordList)
            wordList_UNK = processed_text.split()
            createUnigrams(wordList_UNK,unigram_count)
            createBigrams(wordList_UNK,bigram_count)
            createTrigrams(wordList_UNK,trigram_count)

    generate_ngram_txt(unigram_count,"unigram")
    generate_ngram_txt(bigram_count,"bigram")
    generate_ngram_txt(trigram_count,"trigram")

    print("Testing.......")
    print("========================")
    for f in os.listdir(testPath):
        test_text = ""
        if f.endswith(fileExtension):
            wordList = get_word_list(f)
            test_text = process_file_test(wordList)
        pp = calculate_perplexity(test_text,sum(unigram_count.values()),minLambda)
        print("File : " + f + " Perplexity: " + pp)

In [474]:
trainLM()

Getting word count....9.txt
Getting word count....8.txt
Getting word count....5.txt
Getting word count....4.txt
Getting word count....6.txt
Getting word count....7.txt
Getting word count....3.txt
Getting word count....2.txt
Getting word count....0.txt
Getting word count....1.txt
Writing to file  unigram.txt at ./Output
Writing to file  bigram.txt at ./Output
Writing to file  trigram.txt at ./Output
Testing.......


IOError: [Errno 2] No such file or directory: './train/test01.txt'

In [480]:
print("Testing.......")
print("========================")
for f in os.listdir(testPath):
    test_text = ""
    if f.endswith(fileExtension):
        wordList = get_word_list(testPath,f)
        test_text = process_file_test(wordList)
    pp = calculate_perplexity(test_text,sum(unigram_count.values()),minLambda)
    print("File : " + f + " Perplexity: " + str(pp))

Testing.......
Getting word count....test01.txt
lambda: 0.5 0.4 0.3 Perplexity :318.590021846
File : test01.txt Perplexity: 318.590021846
Getting word count....test02.txt
lambda: 0.5 0.4 0.3 Perplexity :172.305120944
File : test02.txt Perplexity: 172.305120944
